In [93]:
# Libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import pymysql
import glob

##Connect to local DB
driver   = 'mysql+pymysql:'
user     = 'root' 
password = 'ML_LI_04'
ip       = '127.0.0.1' 
database = 'bcn_crime' 

connection_string = f'{driver}//{user}:{password}@{ip}/{database}'
engine = create_engine(connection_string)

### ABP table


In [85]:
## Prepare table

municipis = pd.read_json('https://mossos.gencat.cat/web/.content/home/01_els_mossos_desquadra/indicadors_i_qualitat/Dades_obertes/fitxers/json/Municipis_ABP_2019.json')
municipis.columns = ['year', 'municipality', 'region', 'police_basic_area_abp', 'police_region_rp']
municipis.drop(['year'], axis=1, inplace=True)
municipis_unique = municipis.drop_duplicates()
municipis_unique = municipis_unique[['police_basic_area_abp', 'police_region_rp', 'region', 'municipality']]

In [87]:
print('Len abp before DB: ', len(municipis_unique))

Len abp before DB:  956


In [86]:
municipis_unique.to_sql('abp', con=engine, if_exists = 'append', index = False)

C:\Users\miner\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1366, "Incorrect string value: '\\xE1ndar ...' for column 'VARIABLE_VALUE' at row 1")
  result = self._query(query)


In [103]:
abp_wit_id = pd.read_sql('abp', engine)
print('Len abp after DB: ', len(abp_wit_id))

Len abp after DB:  956


In [104]:
abp_wit_id.head()

,ID,police_basic_area_abp,police_region_rp,region,municipality
0,1,Pallars Jussà-Pallars Sobirà,RP Pirineu Occidental,Pallars Jussà,Abella de la Conca
1,2,Martorell,RP Metropolitana Sud,Baix Llobregat,Abrera
2,3,Noguera,RP Ponent,Noguera,Àger
3,4,Segarra-Urgell,RP Ponent,Urgell,Agramunt
4,5,Bages,RP Central,Bages,Aguilar de Segarra


### Criminality in Catalunya table

In [90]:
##JSONS for the different years
list_jsons = ['https://mossos.gencat.cat/web/.content/home/01_els_mossos_desquadra/indicadors_i_qualitat/Dades_obertes/fitxers/json/Fets_2019.json', 'https://mossos.gencat.cat/web/.content/home/01_els_mossos_desquadra/indicadors_i_qualitat/Dades_obertes/fitxers/json/Fets_2018.json', 'https://mossos.gencat.cat/web/.content/home/01_els_mossos_desquadra/indicadors_i_qualitat/Dades_obertes/fitxers/json/Fets_2017.json', 'https://mossos.gencat.cat/web/.content/home/01_els_mossos_desquadra/indicadors_i_qualitat/Dades_obertes/fitxers/json/Fets_2016.json', 'https://mossos.gencat.cat/web/.content/home/01_els_mossos_desquadra/indicadors_i_qualitat/Dades_obertes/fitxers/json/Fets_2015.json', 'https://mossos.gencat.cat/web/.content/home/01_els_mossos_desquadra/indicadors_i_qualitat/Dades_obertes/fitxers/json/Fets_2014.json', 'https://mossos.gencat.cat/web/.content/home/01_els_mossos_desquadra/indicadors_i_qualitat/Dades_obertes/fitxers/json/Fets_2013.json', 'https://mossos.gencat.cat/web/.content/home/01_els_mossos_desquadra/indicadors_i_qualitat/Dades_obertes/fitxers/json/Fets_2012.json', 'https://mossos.gencat.cat/web/.content/home/01_els_mossos_desquadra/indicadors_i_qualitat/Dades_obertes/fitxers/json/Fets_2011.json']

In [91]:
##Create CSVs with the info
index_csv = 1
for i in list_jsons:
    data = pd.read_json(i)
    data.to_csv(f'Mossos_crimes_{index_csv}.csv')
    index_csv +=1
    

In [94]:
##Import all CSVs in one DF
mossos_crimes_2011_2019 = pd.concat([pd.read_csv(f) for f in glob.glob('*.csv')], ignore_index = True)
mossos_crimes_2011_2019

,Unnamed: 0,Mes,Nom mes,Any,Regió Policial (RP),Àrea Bàsica Policial (ABP),Títol Codi Penal,Tipus de fet,Coneguts,Resolts,Detencions
0,0,1,gener,2019,RP Metropolitana Nord,ABP Arenys de Mar,De l’homicidi i les seves formes,Homicidi consumat,1.0,1.0,NaN
1,1,1,gener,2019,RP Metropolitana Nord,ABP Arenys de Mar,De l’homicidi i les seves formes,Homicidi temptativa,1.0,1.0,1.0
2,2,1,gener,2019,RP Metropolitana Nord,ABP Arenys de Mar,De les falsedats,De la usurpació de l'estat civil,2.0,NaN,NaN
3,3,1,gener,2019,RP Metropolitana Nord,ABP Arenys de Mar,De les falsedats,Falsedats documentals,2.0,1.0,NaN
4,4,1,gener,2019,RP Metropolitana Nord,ABP Arenys de Mar,De les falsedats,Falsificació de moneda i efectes timbrats,2.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
173260,19898,12,desembre,2011,RP Terres de l'Ebre,ABP Terra Alta - Ribera d'Ebre,Delictes contra la llibertat i la indemnitat s...,Abusos sexuals,1.0,2.0,NaN
173261,19899,12,desembre,2011,RP Terres de l'Ebre,ABP Terra Alta - Ribera d'Ebre,Delictes contra l'ordre públic,Altres delictes contra l'ordre públic,NaN,NaN,1.0
173262,19900,12,desembre,2011,RP Terres de l'Ebre,ABP Terra Alta - Ribera d'Ebre,Dels delictes relatius a l’ordenació del terri...,Contra els recursos naturals i el medi ambient,NaN,1.0,NaN
173263,19901,12,desembre,2011,RP Terres de l'Ebre,ABP Terra Alta - Ribera d'Ebre,Faltes contra el patrimoni,Faltes contra el patrimoni,30.0,4.0,NaN


In [95]:
##Drop unwanted columns
mossos_crimes_2011_2019.drop(['Unnamed: 0', 'Resolts', 'Detencions'], axis=1)

,Mes,Nom mes,Any,Regió Policial (RP),Àrea Bàsica Policial (ABP),Títol Codi Penal,Tipus de fet,Coneguts
0,1,gener,2019,RP Metropolitana Nord,ABP Arenys de Mar,De l’homicidi i les seves formes,Homicidi consumat,1.0
1,1,gener,2019,RP Metropolitana Nord,ABP Arenys de Mar,De l’homicidi i les seves formes,Homicidi temptativa,1.0
2,1,gener,2019,RP Metropolitana Nord,ABP Arenys de Mar,De les falsedats,De la usurpació de l'estat civil,2.0
3,1,gener,2019,RP Metropolitana Nord,ABP Arenys de Mar,De les falsedats,Falsedats documentals,2.0
4,1,gener,2019,RP Metropolitana Nord,ABP Arenys de Mar,De les falsedats,Falsificació de moneda i efectes timbrats,2.0
...,...,...,...,...,...,...,...,...
173260,12,desembre,2011,RP Terres de l'Ebre,ABP Terra Alta - Ribera d'Ebre,Delictes contra la llibertat i la indemnitat s...,Abusos sexuals,1.0
173261,12,desembre,2011,RP Terres de l'Ebre,ABP Terra Alta - Ribera d'Ebre,Delictes contra l'ordre públic,Altres delictes contra l'ordre públic,NaN
173262,12,desembre,2011,RP Terres de l'Ebre,ABP Terra Alta - Ribera d'Ebre,Dels delictes relatius a l’ordenació del terri...,Contra els recursos naturals i el medi ambient,NaN
173263,12,desembre,2011,RP Terres de l'Ebre,ABP Terra Alta - Ribera d'Ebre,Faltes contra el patrimoni,Faltes contra el patrimoni,30.0


In [ ]:
##Merge with ABP --- PENDING --- NEED TO NORMALIZE COLUMN

In [98]:
##Export for first analysis
mossos_crimes_2011_2019.to_csv('all_mossos_crimes.csv')

In [ ]:
## Types of crime

In [115]:
types_of_crime = mossos_crimes_2011_2019['Tipus de fet']

In [119]:
unique_crimes = types_of_crime.unique()

In [121]:
df = pd.DataFrame(unique_crimes)

In [123]:
df.to_csv('types_of_crime.csv')